# 1. Data Mining

In this study, data mining was conducted on financial datasets obtained from Eikon, with a specific focus on extracting relevant information related to the S&P 500, London Stock Exchange (LSEG), and their peer companies. However, it should be noted that due to the limitations of the current environment, the code utilized for the data mining process cannot be executed here. Nevertheless, this research paper aims to provide comprehensive insights and analysis based on the mined data from Eikon, utilizing appropriate methodologies and techniques to support the study's objectives.

# S&P500

In [ ]:
import refinitiv.dataplatform.eikon as ek
import numpy as np
import numpy_financial as npf
from numpy import where
from numpy import unique
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LinearRegression
import ipywidgets as widgets
from ipywidgets import Box, Label, Layout
import matplotlib as plt
import warnings

%matplotlib inline
plt.style.use("seaborn")
warnings.filterwarnings("ignore")
ek.set_app_key('DEFAULT_CODE_BOOK_APP_KEY')

In [ ]:
sp500,er = ek.get_data(instruments=".SPX", fields=["TR.IndexConstituentRIC","TR.IndexConstituentName"], parameters={'SDate':'2022-01-01'}
sp500

In [ ]:
rics = sp500['Constituent RIC'].tolist()

## General: name, headquarters, sector, ect

In [ ]:
df_general,err = ek.get_data(rics ,["TR.CommonName(),TR.HeadquartersCountry,TR.TRBCIndustryGroup,TR.GICSSector,TR.CompanyMarketCap.Currency"])
df_general

In [ ]:
df_general.to_csv('sp500_general.csv', index=False)

## Financials: BS, P&L, Cashflow

In [ ]:
df_sp500,err = ek.get_data(rics ,["TR.Revenue.date", "TR.Revenue", "TR.GrossProfit", 'TR.EBIT','TR.EBITDA','TR.F.TotAssets','TR.F.TotLiab',
                                  "TR.EV","TR.BasicEPS", "TR.MarketCap", "TR.FreeCashFlow",
        "TR.PE", "TR.PriceToSales", "TR.PriceToBook",
        "TR.DividendYield", "TR.NetProfitMargin",
        "TR.ReturnOnEquity", "TR.ReturnOnAssets",
        "TR.DebtEquityRatio", "TR.CurrentRatio", "TR.QuickRatio", 'TR.F.TotDebtPctofTotEg','TR.F.DebtTot','TR.WACC','TR.F.ComShrOutsTot(Period=FY0)'],
    {"SDate": 0, "EDate": -20, "FRQ": "FQ", "Curn": "GBP"})
df_sp500

In [ ]:
df_sp500.to_csv('sp500_financials.csv', index=False)

## Transcripts and Press releases

In [ ]:
df_guidance,err = ek.get_data(rics,['TR.GuidanceDate','TR.GuidanceMeasure','TR.EstGuidHighValue','TR.EstGuidLowValue','TR.GuidanceText','TR.GuidanceDocType'],
                   {'Period':'FY1','GuidMeasure':'REV,EBIT,EBITDA','GuidDataBasis':'VAL'})

df_guidance

In [ ]:
df_guidance.to_csv('sp500_text.csv', index=False)

## Target variables

In [ ]:
#target varibales

df_target,err = ek.get_data(rics,['TR.FwdEVToEBITDA','TR.EVToEBITDA', 'TR.EVtoFCFSmartEst(Period=FY1)','TR.PE','TR.FwdPE'])
df_target

In [ ]:
df_targetv.to_csv('sp500_target_variables.csv', index=False)

# LSEG, Peers

In [ ]:
RICs = ['LSEG.L', 'MCO', 'MORN.O', 'SPGI.K', 'MSCI.K', 'VRSK.O', 'FDS','ICE', 'NDAQ.O', 'EEFT.O', 'CME.O', 'CBOE.K','0388.HK','DB1GnEUR.xbo' ,'MKTX.O']

In [ ]:
#general
df_general,err = ek.get_data(RICs,["TR.CommonName(),TR.HeadquartersCountry,TR.TRBCIndustryGroup,TR.GICSSector,TR.CompanyMarketCap.Currency"])

#financials

df_financials,err = ek.get_data(RICs ,["TR.Revenue.date", "TR.Revenue", "TR.GrossProfit", 'TR.EBIT','TR.EBITDA','TR.F.TotAssets','TR.F.TotLiab',
                                  "TR.EV","TR.BasicEPS", "TR.MarketCap", "TR.FreeCashFlow",
        "TR.PE", "TR.PriceToSales", "TR.PriceToBook",
        "TR.DividendYield", "TR.NetProfitMargin",
        "TR.ReturnOnEquity", "TR.ReturnOnAssets",
        "TR.DebtEquityRatio", "TR.CurrentRatio", "TR.QuickRatio", 'TR.F.TotDebtPctofTotEg','TR.F.DebtTot','TR.WACC','TR.F.ComShrOutsTot(Period=FY0)'],
    {"SDate": 0, "EDate": -20, "FRQ": "FQ", "Curn": "GBP"})


# transcripts and press realeases

df_guidance,err = ek.get_data(RICs,['TR.GuidanceDate','TR.GuidanceMeasure','TR.EstGuidHighValue','TR.EstGuidLowValue','TR.GuidanceText','TR.GuidanceDocType'],
                   {'Period':'FY1','GuidMeasure':'REV,EBIT,EBITDA','GuidDataBasis':'VAL'})

#target varibales

df_target,err = ek.get_data(RICs,['TR.FwdEVToEBITDA','TR.EVToEBITDA', 'TR.EVtoFCFSmartEst(Period=FY1)','TR.PE','TR.FwdPE'])

In [ ]:
#to csv

df_general.to_csv('lseg_general.csv', index=False)
df_financials.to_csv('lseg_financials.csv', index=False)
df_guidance.to_csv('lseg_text.csv', index=False)
df_target.to_csv('lseg_target_variables.csv', index=False)

# World Bank

Adding Macro-economic features

- gdp
- inflation
- undemployment
- exchange rate
- dept % of gdp
- stock traded % of gpd
- balance of trade
- fiscal policy expeditures

## S&P 500

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
df_general = pd.read_csv('/content/drive/MyDrive/Dissertation_Final /Data/Raw/sp500_general.csv')

# Get the count of unique countries
unique_counts = df_general['Country of Headquarters'].value_counts()
for value, count in unique_counts.items():
    print(value, count) # HKG ,DEU,USA,GBR

United States of America 483
Ireland; Republic of 11
United Kingdom 7
Switzerland 3
Israel 1
Netherlands 1
Bermuda 1


United States of America 12
United Kingdom 1
Hong Kong 1
Germany 1


In [ ]:
!pip install wbgapi

In [ ]:
!pip install wbdata

In [ ]:
import wbdata
wbdata.get_country()

#wb.source.info()
#wb.series.info(db = 2)
#wbdata.get_topic()

id    name
----  --------------------------------------------------------------------------------
ABW   Aruba
AFE   Africa Eastern and Southern
AFG   Afghanistan
AFR   Africa
AFW   Africa Western and Central
AGO   Angola
ALB   Albania
AND   Andorra
ARB   Arab World
ARE   United Arab Emirates
ARG   Argentina
ARM   Armenia
ASM   American Samoa
ATG   Antigua and Barbuda
AUS   Australia
AUT   Austria
AZE   Azerbaijan
BDI   Burundi
BEA   East Asia & Pacific (IBRD-only countries)
BEC   Europe & Central Asia (IBRD-only countries)
BEL   Belgium
BEN   Benin
BFA   Burkina Faso
BGD   Bangladesh
BGR   Bulgaria
BHI   IBRD countries classified as high income
BHR   Bahrain
BHS   Bahamas, The
BIH   Bosnia and Herzegovina
BLA   Latin America & the Caribbean (IBRD-only countries)
BLR   Belarus
BLZ   Belize
BMN   Middle East & North Africa (IBRD-only countries)
BMU   Bermuda
BOL   Bolivia
BRA   Brazil
BRB   Barbados
BRN   Brunei Darussalam
BSS   Sub-Saharan Africa (IBRD-only countries)
BTN   Bhutan
BWA  

In [ ]:
import wbgapi as wb
import pandas as pd

trial = wb.data.DataFrame('NY.GDP.PCAP.CD',['USA','IRL','BMU','NLD','ISR','CHE','GBR'],range(2016,2023))
trial = trial.reset_index()
trial.columns = [col.replace('YR', '') for col in trial.columns]
df1 = trial.melt(id_vars='economy', var_name='Year', value_name='GDP')
df1.head(5) # this GDP

,economy,Year,GDP
0,BMU,2016,106885.878489
1,CHE,2016,82153.074545
2,GBR,2016,41146.077356
3,IRL,2016,62895.911124
4,ISR,2016,37690.473951


In [ ]:
import wbgapi as wb
import pandas as pd

def get_time_series_data(indicator_code, countries, year_range, value_name):
    # Retrieve the data using wbgapi
    data = wb.data.DataFrame(indicator_code, countries, year_range)

    # Rename the columns
    data = data.reset_index()
    data.columns = [col.replace('YR', '') for col in data.columns]

    # Melt the DataFrame to convert it into a time series format
    time_series_df = data.melt(id_vars='economy', var_name='Year', value_name=value_name)

    return time_series_df

# Example usage
indicator_code = 'CM.MKT.TRAD.GD.ZS'
countries = ['USA', 'IRL', 'BMU', 'NLD', 'ISR', 'CHE', 'GBR']
year_range = range(2016, 2023)
value_name = 'stock traded percentage of gpd'

# Call the function to retrieve the time series data
df2 = get_time_series_data(indicator_code, countries, year_range, value_name)

# Print the resulting time series DataFrame
df2.head(5)

,economy,Year,stock traded percentage of gpd
0,BMU,2016,3.144533
1,CHE,2016,121.710067
2,GBR,2016,NaN
3,IRL,2016,7.471138
4,ISR,2016,16.089230


In [ ]:
# Inflation
indicator_code = 'FP.CPI.TOTL.ZG'
value_name = 'Inflation, consumer prices annual percentage'
df3 = get_time_series_data(indicator_code, countries, year_range, value_name)
df3.head(5)

# Unemployment
indicator_code = 'SL.UEM.TOTL.NE.ZS'
value_name = 'Unemployment total (pcg of total labor force) (national estimate)'
df4 = get_time_series_data(indicator_code, countries, year_range, value_name)
df4.head(5)

# DEBT
indicator_code = 'GC.DOD.TOTL.GD.ZS'
value_name = ' General government gross debt (% of GDP)'

df5 = get_time_series_data(indicator_code, countries, year_range, value_name)
df5.head(5)

,economy,Year,General government gross debt (% of GDP)
0,BMU,2016,NaN
1,CHE,2016,19.380993
2,GBR,2016,156.004943
3,IRL,2016,85.332512
4,ISR,2016,NaN


In [ ]:
# Balance of trade
indicator_code = 'BN.GSR.GNFS.CD'
value_name = 'Balance of Trade'

df6 = get_time_series_data(indicator_code, countries, year_range, value_name)
df6.head(5)

,economy,Year,Balance of Trade
0,BMU,2016,-6.104830e+08
1,CHE,2016,6.247925e+10
2,GBR,2016,-5.270381e+10
3,IRL,2016,4.690930e+10
4,ISR,2016,6.004200e+09


In [ ]:
# EXCHANGE RATe
indicator_code = 'PA.NUS.FCRF'
value_name = 'Official exchange rate (LCU per US$, period average)'

df7 = get_time_series_data(indicator_code, countries, year_range, value_name)
df7.head(5)

,economy,Year,"Official exchange rate (LCU per US$, period average)"
0,BMU,2016,1.000000
1,CHE,2016,0.985394
2,GBR,2016,0.740634
3,IRL,2016,0.903421
4,ISR,2016,3.840567


In [ ]:
# Fiscal Policy.
indicator_code = 'NE.CON.GOVT.ZS'
value_name = 'General government final consumption expenditure (% of GDP)'

df8 = get_time_series_data(indicator_code, countries, year_range, value_name)
df8.head(5)

,economy,Year,General government final consumption expenditure (% of GDP)
0,BMU,2016,12.130591
1,CHE,2016,11.514962
2,GBR,2016,19.133456
3,IRL,2016,12.765646
4,ISR,2016,21.930218


In [ ]:
def merge_dataframes(dfs):
    # Start with the first dataframe in the list
    df_merged = dfs[0]

    # Loop over the rest of the dataframes and merge them one by one
    for df in dfs[1:]:
        df_merged = df_merged.merge(df, on=['economy', 'Year'], how='inner')

    return df_merged

In [ ]:
# List all your dataframes
dfs = [df1, df2, df3, df4,df5,df6,df7,df8]
# Call the function to merge the dataframes
df_macro= merge_dataframes(dfs)

In [ ]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Instantiate the IterativeImputer
imputer = IterativeImputer()

# Select the columns you want to impute
cols_to_impute = df_macro.columns.drop('economy')

# Apply the imputer to these columns
df_macro[cols_to_impute] = imputer.fit_transform(df_macro[cols_to_impute])

In [ ]:
# Create a dictionary mapping codes to names
country_dict = {
    'USA': 'United States of America',
    'IRL': 'Republic of Ireland',
    'GBR': 'United Kingdom',
    'CHE': 'Switzerland',
    'ISR': 'Israel',
    'NLD': 'Netherlands',
    'BMU': 'Bermuda'
}

# Use the replace function to replace the country codes with names
df_macro['economy'] = df_macro['economy'].replace(country_dict)


df_macro = df_macro.rename(columns={'economy': 'Country'})

In [ ]:
df_macro.head(5)

,Country,Year,GDP,stock traded percentage of gpd,"Inflation, consumer prices annual percentage",Unemployment total (pcg of total labor force) (national estimate),General government gross debt (% of GDP),Balance of Trade,"Official exchange rate (LCU per US$, period average)",General government final consumption expenditure (% of GDP)
0,Bermuda,2016.0,106885.878489,3.144533,2.293607,5.838422,-13.427599,-6.104830e+08,1.000000,12.130591
1,Switzerland,2016.0,82153.074545,121.710067,-0.434619,4.920000,19.380993,6.247925e+10,0.985394,11.514962
2,United Kingdom,2016.0,41146.077356,57.107859,1.008417,4.810000,156.004943,-5.270381e+10,0.740634,19.133456
3,Republic of Ireland,2016.0,62895.911124,7.471138,0.008306,8.370000,85.332512,4.690930e+10,0.903421,12.765646
4,Israel,2016.0,37690.473951,16.089230,-0.551552,4.720000,171.625983,6.004200e+09,3.840567,21.930218


In [ ]:
df_macro.to_csv('sp500_macro.csv')

## Lseg and peers

the countries are:
- United States of America 12
- United Kingdom 1
- Hong Kong 1
- Germany 1

In [ ]:
import wbgapi as wb
import pandas as pd

trial = wb.data.DataFrame('NY.GDP.PCAP.CD',['HKG' ,'DEU','USA','GBR'],range(2016,2023))
trial = trial.reset_index()
trial.columns = [col.replace('YR', '') for col in trial.columns]
df1 = trial.melt(id_vars='economy', var_name='Year', value_name='GDP')
df1.head(5) # this GDP

,economy,Year,GDP
0,DEU,2016,42136.120791
1,GBR,2016,41146.077356
2,HKG,2016,43734.198070
3,USA,2016,57866.744934
4,DEU,2017,44652.589172


In [ ]:
import wbgapi as wb
import pandas as pd

def get_time_series_data(indicator_code, countries, year_range, value_name):
    # Retrieve the data using wbgapi
    data = wb.data.DataFrame(indicator_code, countries, year_range)

    # Rename the columns
    data = data.reset_index()
    data.columns = [col.replace('YR', '') for col in data.columns]

    # Melt the DataFrame to convert it into a time series format
    time_series_df = data.melt(id_vars='economy', var_name='Year', value_name=value_name)

    return time_series_df

# Example usage
indicator_code = 'CM.MKT.TRAD.GD.ZS'
countries = ['HKG' ,'DEU','USA','GBR']
year_range = range(2016, 2023)
value_name = 'stock traded percentage of gpd'

# Call the function to retrieve the time series data
df2 = get_time_series_data(indicator_code, countries, year_range, value_name)

# Print the resulting time series DataFrame
df2.head(5)

,economy,Year,stock traded percentage of gpd
0,DEU,2016,32.396521
1,GBR,2016,NaN
2,HKG,2016,421.085788
3,USA,2016,225.039211
4,DEU,2017,42.228739


In [ ]:
# Inflation
indicator_code = 'FP.CPI.TOTL.ZG'
value_name = 'Inflation, consumer prices annual percentage'
df3 = get_time_series_data(indicator_code, countries, year_range, value_name)

# Unemployment
indicator_code = 'SL.UEM.TOTL.NE.ZS'
value_name = 'Unemployment total (pcg of total labor force) (national estimate)'
df4 = get_time_series_data(indicator_code, countries, year_range, value_name)

# DEBT
indicator_code = 'GC.DOD.TOTL.GD.ZS'
value_name = ' General government gross debt (% of GDP)'
df5 = get_time_series_data(indicator_code, countries, year_range, value_name)

# Balance of trade
indicator_code = 'BN.GSR.GNFS.CD'
value_name = 'Balance of Trade'
df6 = get_time_series_data(indicator_code, countries, year_range, value_name)

# EXCHANGE RATe
indicator_code = 'PA.NUS.FCRF'
value_name = 'Official exchange rate (LCU per US$, period average)'
df7 = get_time_series_data(indicator_code, countries, year_range, value_name)

# Fiscal Policy.
indicator_code = 'NE.CON.GOVT.ZS'
value_name = 'General government final consumption expenditure (% of GDP)'
df8 = get_time_series_data(indicator_code, countries, year_range, value_name)

In [ ]:
def merge_dataframes(dfs):
    # Start with the first dataframe in the list
    df_merged = dfs[0]

    # Loop over the rest of the dataframes and merge them one by one
    for df in dfs[1:]:
        df_merged = df_merged.merge(df, on=['economy', 'Year'], how='inner')

    return df_merged

# List all your dataframes
dfs = [df1, df2, df3, df4,df5,df6,df7,df8]
# Call the function to merge the dataframes
df_macro= merge_dataframes(dfs)

In [ ]:
# Create a dictionary mapping codes to names
country_dict = {
    'USA': 'United States of America',
    'GBR': 'United Kingdom',
    'HKG': 'Hong Kong',
    'DEU': 'Germany',
}

# Use the replace function to replace the country codes with names
df_macro['economy'] = df_macro['economy'].replace(country_dict)
df_macro = df_macro.rename(columns={'economy': 'Country'})

In [ ]:
df_macro.head(3)

,Country,Year,GDP,stock traded percentage of gpd,"Inflation, consumer prices annual percentage",Unemployment total (pcg of total labor force) (national estimate),General government gross debt (% of GDP),Balance of Trade,"Official exchange rate (LCU per US$, period average)",General government final consumption expenditure (% of GDP)
0,Germany,2016,42136.120791,32.396521,0.491747,4.12,NaN,2.564020e+11,0.903421,19.901204
1,United Kingdom,2016,41146.077356,NaN,1.008417,4.81,156.004943,-5.270381e+10,0.740634,19.133456
2,Hong Kong,2016,43734.198070,421.085788,2.409307,3.39,NaN,7.327349e+09,7.762250,9.956364


In [ ]:
df_macro.to_csv('lseg_macro.csv')